# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import string
import pickle

import pandas as pd
import numpy as np

from sqlalchemy import create_engine

import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("Message", engine)
df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
nltk.download(['punkt', 'wordnet', 'stopwords'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import re

skip_tokens = stopwords.words('english') + ["'s", "n't"] + list(string.punctuation)
def tokenize(text):
    lemm = WordNetLemmatizer()
    
    text = word_tokenize(text)
    text = [lemm.lemmatize(x).lower().strip() for x in text]
    text = filter(lambda x: x not in skip_tokens, text)
    
    return " ".join(text)

tokenize("Hello, My name wouldn't Heegyu Kim. He's name isn't perfectly perfects but that's ok and so on...")


'hello name would heegyu kim name perfectly perfect ok ...'

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline =  Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X = df.message
Y = df.iloc[:, 5:]

X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=2)


In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns
y_pred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
def test_prediction():
    y_pred = pipeline.predict(X_test)
    y_pred = pd.DataFrame(y_pred)
    y_pred.columns = y_test.columns
    
    reports = []
    for column in Y.columns:
        report = classification_report(y_test[column], y_pred[column])
        reports.append(report)

        print("Classification report for {}".format(column))
        print(report)
test_prediction()

Classification report for related
             precision    recall  f1-score   support

          0       0.47      0.26      0.34      1977
          1       0.80      0.91      0.85      6616
          2       0.43      0.05      0.09        59

avg / total       0.72      0.76      0.73      8652

Classification report for request
             precision    recall  f1-score   support

          0       0.85      0.98      0.91      7207
          1       0.63      0.15      0.24      1445

avg / total       0.81      0.84      0.80      8652

Classification report for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8615
          1       0.00      0.00      0.00        37

avg / total       0.99      1.00      0.99      8652

Classification report for aid_related
             precision    recall  f1-score   support

          0       0.65      0.80      0.72      5110
          1       0.57      0.37      0.45      3542

avg

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {
#     'clf__estimator__max_depth': [10, 25, 50],
#     'clf__estimator__n_estimators': [10, 50, 100],

#     'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (5000, 10000, 50000)
}

cv = GridSearchCV(pipeline, parameters, verbose=1)

In [14]:
cv.fit(X_train, y_train)


Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  7.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_features': (5000, 10000, 50000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [15]:
print('final params', cv.best_params_)   # best parameters
print('best score', cv.best_score_)      # best score

final params {'vect__max_features': 50000}
best score 0.183728080164


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
test_prediction()

Classification report for related
             precision    recall  f1-score   support

          0       0.47      0.26      0.34      1977
          1       0.80      0.91      0.85      6616
          2       0.43      0.05      0.09        59

avg / total       0.72      0.76      0.73      8652

Classification report for request
             precision    recall  f1-score   support

          0       0.85      0.98      0.91      7207
          1       0.63      0.15      0.24      1445

avg / total       0.81      0.84      0.80      8652

Classification report for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8615
          1       0.00      0.00      0.00        37

avg / total       0.99      1.00      0.99      8652

Classification report for aid_related
             precision    recall  f1-score   support

          0       0.65      0.80      0.72      5110
          1       0.57      0.37      0.45      3542

avg

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [17]:
with open("model.pkt", "wb") as f:
    pickle.dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.